In [1]:
import pandas as pd
import sqlalchemy as db
import cx_Oracle as orcl
import datetime as dt
import numpy as np
import psycopg2 
import seaborn as sns
import matplotlib.pyplot as plt
import math
import datetime as dt
%matplotlib inline
print("cx_Oracle.version:", orcl.version)
# show all columns without without truncating
pd.set_option('display.max_columns', None)



cx_Oracle.version: 7.3.0


conn = psycopg2.connect(host="128.2.1.25",database="datawarehouse", user="datawarehouse", password="P@ssw0rd")

cur = conn.cursor()

In [2]:
#cur.close()

In [3]:
def status_score(x):
    if x == 20:
        return 20
    elif x == -1000:
        return -1000
    else:
        return 0
    

In [4]:
def status_category(x):
    if x == 'good':
        return 20
    elif x == 'bad':
        return -1000
    else:
        return 0

In [5]:
def age_score(x):
    if x <= 9125.0:
        return -1000
    elif x <= 16000.0:
        return 5
    elif x <= 19000.0: 
        return 10
    elif x > 19000.0: 
        return 20
    else:
        return 0

In [6]:
def frequency_score(x):
    if x <= 8:
        return -10
    elif x <= 24:
        return 5
    elif x <= 48: 
        return 10
    elif x > 48: 
        return 30
    
    else:
        return 0

In [7]:
def transaction_amount_sum_score(x):
    if x <= 1000:
        return -10
    elif x <= 10000:
        return 5
    elif x <= 30000: 
        return 10
    elif x > 30000: 
        return 15
    else:
        return 0

In [8]:
def recency_score(x):
    if x <= 5:
        return 20
    elif x <= 10:
        return 10
    elif x <= 30: 
        return 5
    elif x > 30: 
        return 1
    else:
        return 0

In [9]:
def age_of_acccount_in_bank_score(x):
    if x <= 45:
        return -10
    elif x <= 110:
        return 5
    elif x <= 240: 
        return 10
    elif x > 240: 
        return 15
    
    else:
        return 0

In [10]:
def count_of_loans_taken_score(x):
    if x <= 1:
        return 1
    elif x <= 10:
        return 5
    elif x <= 20: 
        return 10
    elif x > 20: 
        return 10
    else:
        return 0

In [11]:
def limit_mutiplier(x):
    if x <= 30:
        return 0
    elif x <= 60:
        return 0.5
    elif x >60: 
        return 1
    
    else:
        return 0

In [12]:
def limit_ratified(x):
    if x <= 1000:
        return 0
    elif x <= 30000:
        return x
    elif x >=30000: 
        return 30000
    else:
        return 0

In [13]:
def currency_ratified_score(x):
    if x == 'KES':
        return 1
    elif x in ('USD', 'GBP', 'EUR'):
        return 0
    else:
        return 0

In [14]:
#giving account which are joint zero limit
def joint_account_status_ratified(x):
    if x == '1':
        return 1
    elif x == '2':
        return 0
    else:
        return 0

In [15]:
#removing under 25
def removing_under_25(x):
    if x <=25:
        return 0
    else:
        return 0

In [16]:
def account_status_ratified(x):
    if x in ('ACTIVE', 'LOCKED'):
        return 1
    elif x in ( 'DORMANT', 'BLOCKED'):
        return 0
    else:
        return 0

In [17]:
def remove_non_transacting_accounts(x):
    if x == True :
        return 1
    elif x == False:
        return 0
    else:
        return 0

In [18]:
def remove_non_qualifying_products(x):
    if x in('VIRTUAL ACCOUNT MOBILE', 'HFC SALARY ACCOUNT', 'PERSONAL CURRENT ACCOUNT-UNBUN','LENGO ACCOUNT', 'SME BIZ ACCOUNT','TRANSACTIONAL ACCOUNT-CA', 'MICRO BIZ ACCOUNT', 'PERSONAL CURRENT ACCOUNT BUND','YOLO ACCOUNT', 'HFC KILIMO ACCOUNT', 'EASY COLLECT ACCOUNT') :
        return 1
    elif x in ('CROSS OVER SAVING ACCOUNT', 'SAVINGS USD', 'TREASURE SAVINGS ACCOUNT',  'KUNDI BORA ACCOUNT', '1ST HOP',  'REGISTRAR HIGH COURT SAVINGS A', 'TAKE ON ACCOUNT', 'CURRENT USD', 'CURRENT ACCOUNT STAFF', 'SAVINGS GBP', 'STAFF SALARY ACCOUNT', 'MEMO PRODUCT-JOURNAL', 'ESCROW CURRENT ACCOUNT', 'MAKAO SAVING ACCOUNT', 'CURRENT GBP', 'STAFF SAVINGS ACCOUNT', 'SUBSCRIPTION BOND SAVINGS ACCO', 'MORTGAGE SCHEME DEP-QUARTERLY INTEREST', 'INSTITUTIONAL BANKING-CA', 'CURRENT EURO', 'SAVINGS EURO', 'MORTGAGE SCHEME DEPOSIT', 'CURRENT ACCOUNT CORPORATE','FLEXI ACCOUNT','APEX ACCOUNT',) :
        return 0
    else:
        return 0

In [19]:
def remove_write_off_accounts(x):
    if x == 1.0 :
        return 0
    else:
        return 1

In [20]:
def removing_unqualifying_segments(x):
    if x in ('VIRTUAL', 'MASS',  'STANDARD', 'PRIVATE'):
        return 1
    elif x in ( 'MEDIUM ENTERPRISES', 'LARGE ENTERPRISES','SMALL ENTERPRISES'):
        return 0
    else:
        return 0

In [21]:
def phone_number_check(x):
    if x ==  12:
        return 1
    else:
        return 0

customers_dump = pd.read_csv("/Users/allan.aswani/Downloads/calculation/dean_data_final/customers_dump_feb_2020.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
customers_dump

customers_dump.dtypes

customers_dump['dob'] = pd.to_datetime(customers_dump['dob'],infer_datetime_format=True , errors='coerce') #, errors='coerce'
customers_dump['open_date'] = pd.to_datetime(customers_dump['open_date'],infer_datetime_format=True ,errors='coerce') #, errors='coerce'

transaction_data = pd.read_csv("/Users/allan.aswani/Downloads/calculation/dean_data_final/dump_transactions_2019_to_date.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
transaction_data

In [22]:
conn = psycopg2.connect(host="128.2.1.25",database="datawarehouse", user="datawarehouse", password="P@ssw0rd")

In [23]:
transaction_data = '''SELECT "public"."transactional_table_attempt_1"."id" AS "id", "public"."transactional_table_attempt_1"."account_name" AS "account_name", "public"."transactional_table_attempt_1"."account_number" AS "account_number", "public"."transactional_table_attempt_1"."commission" AS "commission", "public"."transactional_table_attempt_1"."datecreated" AS "datecreated", "public"."transactional_table_attempt_1"."fk_customercust_id" AS "fk_customercust_id", "public"."transactional_table_attempt_1"."i_id_justific" AS "i_id_justific", "public"."transactional_table_attempt_1"."id_product" AS "id_product", "public"."transactional_table_attempt_1"."mobile_tel" AS "mobile_tel", "public"."transactional_table_attempt_1"."national_id" AS "national_id", "public"."transactional_table_attempt_1"."product_description" AS "product_description", "public"."transactional_table_attempt_1"."reverse_flag" AS "reverse_flag", "public"."transactional_table_attempt_1"."reversed_trx_flag" AS "reversed_trx_flag", "public"."transactional_table_attempt_1"."tmstamp" AS "tmstamp", "public"."transactional_table_attempt_1"."transaction_amount" AS "transaction_amount", "public"."transactional_table_attempt_1"."transaction_ref" AS "transaction_ref", "public"."transactional_table_attempt_1"."transaction_type" AS "transaction_type", "public"."transactional_table_attempt_1"."trx_comments" AS "trx_comments", "public"."transactional_table_attempt_1"."trx_date" AS "trx_date", "public"."transactional_table_attempt_1"."trx_unit" AS "trx_unit", "public"."transactional_table_attempt_1"."trx_usr" AS "trx_usr", "public"."transactional_table_attempt_1"."trx_usr_sn" AS "trx_usr_sn", "public"."transactional_table_attempt_1"."tun_internal_sn" AS "tun_internal_sn"
FROM "public"."transactional_table_attempt_1"'''
transaction_data = pd.read_sql_query(transaction_data , conn)
transaction_data

,id,account_name,account_number,commission,datecreated,fk_customercust_id,i_id_justific,id_product,mobile_tel,national_id,product_description,reverse_flag,reversed_trx_flag,tmstamp,transaction_amount,transaction_ref,transaction_type,trx_comments,trx_date,trx_unit,trx_usr,trx_usr_sn,tun_internal_sn
0,2379331,LIKKI FOODS & CATERING SUPPLIES ...,7413003201,60.0,2020-02-03 16:57:14.830568,741300,32099,31561,,BN-7ZCQQ5B,SME BIZ ACCOUNT,0,0,2019-10-25 16:50:55,6000.0,20191025109KOCE0001120091,ACCOUNT TO MPESA(B2C),SEND TO MPESA 254740877187,2019-10-25,109,KOCE0001,12010,1
1,2379332,LIKKI FOODS & CATERING SUPPLIES ...,7413003201,60.0,2020-02-03 16:57:14.830568,741300,32099,31561,,BN-7ZCQQ5B,SME BIZ ACCOUNT,0,0,2019-10-28 10:47:47,6000.0,20191028109KOCE0001165401,ACCOUNT TO MPESA(B2C),SEND TO MPESA 254714078207,2019-10-28,109,KOCE0001,16541,1
2,2379333,LIKKI FOODS & CATERING SUPPLIES ...,7413003201,30.0,2020-02-03 16:57:14.830568,741300,32099,31561,,BN-7ZCQQ5B,SME BIZ ACCOUNT,0,0,2019-10-28 11:33:37,1000.0,20191028109KOCE0001172481,ACCOUNT TO MPESA(B2C),SEND TO MPESA 254716103725,2019-10-28,109,KOCE0001,17249,1
3,2379334,LIKKI FOODS & CATERING SUPPLIES ...,7413003201,60.0,2020-02-03 16:57:14.830568,741300,32099,31561,,BN-7ZCQQ5B,SME BIZ ACCOUNT,0,0,2019-10-31 05:56:16,8493.0,20191031109KOCE000123951,ACCOUNT TO MPESA(B2C),SEND TO MPESA 254719821840,2019-10-31,109,KOCE0001,2396,1
4,2379335,LIKKI FOODS & CATERING SUPPLIES ...,7413003201,60.0,2020-02-03 16:57:14.830568,741300,32099,31561,,BN-7ZCQQ5B,SME BIZ ACCOUNT,0,0,2019-11-01 12:01:00,5000.0,20191101109KOCE000174341,ACCOUNT TO MPESA(B2C),SEND TO MPESA 254740877187,2019-11-01,109,KOCE0001,7435,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165783,2379326,LIKKI FOODS & CATERING SUPPLIES ...,7413003201,60.0,2020-02-03 16:57:14.830568,741300,32099,31561,,BN-7ZCQQ5B,SME BIZ ACCOUNT,0,0,2019-10-14 10:51:54,7000.0,20191014109KOCE0001124531,ACCOUNT TO MPESA(B2C),SEND TO MPESA 254714078207,2019-10-14,109,KOCE0001,12454,1
1165784,2379327,LIKKI FOODS & CATERING SUPPLIES ...,7413003201,40.0,2020-02-03 16:57:14.830568,741300,32099,31561,,BN-7ZCQQ5B,SME BIZ ACCOUNT,0,0,2019-10-19 12:11:35,2500.0,20191019109KOCE00023651,ACCOUNT TO MPESA(B2C),SEND TO MPESA 254740877187,2019-10-19,109,KOCE0002,366,1
1165785,2379328,LIKKI FOODS & CATERING SUPPLIES ...,7413003201,54.0,2020-02-03 16:57:14.830568,741300,32099,31561,,BN-7ZCQQ5B,SME BIZ ACCOUNT,0,0,2019-10-19 11:56:24,2800.0,20191019109KOCE000144831,ACCOUNT TO MPESA(B2C),SEND TO MPESA 254718603806,2019-10-19,109,KOCE0001,4484,1
1165786,2379329,LIKKI FOODS & CATERING SUPPLIES ...,7413003201,54.0,2020-02-03 16:57:14.830568,741300,32099,31561,,BN-7ZCQQ5B,SME BIZ ACCOUNT,0,0,2019-10-19 12:03:06,2800.0,20191019109KOCE000145941,ACCOUNT TO MPESA(B2C),SEND TO MPESA 254718326736,2019-10-19,109,KOCE0001,4595,1


In [24]:
transaction_data['tmstamp'] = pd.to_datetime(transaction_data['tmstamp'],infer_datetime_format=True )

In [25]:
transaction_data['transaction_amount']=transaction_data['transaction_amount'].astype(float)

In [26]:
transaction_data['transaction_amount'].sum()

12277606056.13

In [27]:
NOW = transaction_data.tmstamp.max()
NOW

Timestamp('2020-03-04 20:10:30')

In [28]:
rfmTable = transaction_data.groupby('account_number').agg({'tmstamp': lambda x: (NOW - x.max()).days, 'transaction_ref': lambda x: len(x), 'transaction_amount': lambda x: x.sum()})
rfmTable['tmstamp'] = rfmTable['tmstamp'].astype(int)
rfmTable.rename(columns={'tmstamp': 'recency', 
                         'transaction_ref': 'frequency', 
                         'transaction_amount': 'monetary_value'}, inplace=True)


In [29]:
rfmTable.reset_index(inplace=True)

In [30]:
rfmTable

,account_number,recency,frequency,monetary_value
0,0000430004,366,1,0.0
1,0000430007,33,1,0.0
2,0000430015,177,1,0.0
3,0000430018,301,1,0.0
4,0002963201,249,1,0.0
...,...,...,...,...
127152,9783692776,0,1,15291.0
127153,9783692850,0,1,6830.0
127154,9783692868,0,3,2820.0
127155,9783692944,0,2,16570.0


 customers_dump.columns

In [31]:
customers_dump = '''SELECT "public"."customers"."id" AS "id", "public"."customers"."firstname" AS "firstname", "public"."customers"."lastname" AS "lastname", "public"."customers"."branch" AS "branch", "public"."customers"."cust_id" AS "cust_id", "public"."customers"."dob" AS "dob", "public"."customers"."email" AS "email", "public"."customers"."gender" AS "gender", "public"."customers"."last_updated_date" AS "last_updated_date", "public"."customers"."middlename" AS "middlename", "public"."customers"."national_id" AS "national_id", "public"."customers"."open_date" AS "open_date", "public"."customers"."other_phone_numbers" AS "other_phone_numbers", "public"."customers"."phone_number" AS "phone_number", "public"."customers"."segment" AS "segment", "public"."customers"."status" AS "status", "public"."customers"."type" AS "type", "Accounts"."id" AS "id_2", "Accounts"."account_name" AS "account_name", "Accounts"."account_no" AS "account_no", "Accounts"."account_status" AS "account_status", "Accounts"."changed_by" AS "changed_by", "Accounts"."close_date" AS "close_date", "Accounts"."currency" AS "currency", "Accounts"."current_balance" AS "current_balance", "Accounts"."current_branch" AS "current_branch", "Accounts"."cust_id" AS "cust_id_2", "Accounts"."date_created" AS "date_created", "Accounts"."eom_date" AS "eom_date", "Accounts"."expiry_date" AS "expiry_date", "Accounts"."interest" AS "interest", "Accounts"."interest_end_date" AS "interest_end_date", "Accounts"."interest_rate" AS "interest_rate", "Accounts"."interest_start_date" AS "interest_start_date", "Accounts"."is_transacting_account" AS "is_transacting_account", "Accounts"."last_transaction_date" AS "last_transaction_date", "Accounts"."open_date" AS "open_date_2", "Accounts"."opened_by" AS "opened_by", "Accounts"."opening_branch" AS "opening_branch", "Accounts"."product_type" AS "product_type", "Accounts"."type" AS "type_2"
FROM "public"."customers"
LEFT JOIN "public"."accounts" "Accounts" ON "public"."customers"."cust_id" = "Accounts"."cust_id"
'''
customers_dump = pd.read_sql_query(customers_dump , conn)
customers_dump

,id,firstname,lastname,branch,cust_id,dob,email,gender,last_updated_date,middlename,national_id,open_date,other_phone_numbers,phone_number,segment,status,type,id_2,account_name,account_no,account_status,changed_by,close_date,currency,current_balance,current_branch,cust_id_2,date_created,eom_date,expiry_date,interest,interest_end_date,interest_rate,interest_start_date,is_transacting_account,last_transaction_date,open_date_2,opened_by,opening_branch,product_type,type_2
0,1684944,HUDSON,MARIGA ...,HEAD OFFICE,103265,1980-04-13 00:00:00+03:00,...,M,0001-01-01 00:00:00+02:27,GACHERU,21982968,2016-01-01 00:00:00+03:00,,254722342828,MASS,1,1,35740180.0,HUDSON MARIGA,2000068873,DORMANT,None,2001-01-01,KES,0.00,NYERI,103265.0,2020-03-05 07:02:01.575886,2020-03-04,2001-01-01,0.0,2001-01-01,0.0,2001-01-01,True,2014-06-10,2008-06-30,None,NYERI,TRANSACTIONAL ACCOUNT-CA,TRANSACTIONAL ACCOUNT-CA
1,1684570,RICHARD,OMWANCHA ...,REHANI,102361,1971-01-01 00:00:00+03:00,...,M,0001-01-01 00:00:00+02:27,NYAMARI,10929252,2016-01-01 00:00:00+03:00,,254723291492,MASS,1,1,NaN,None,None,None,None,NaT,None,NaN,None,NaN,NaT,NaT,NaT,NaN,NaT,NaN,NaT,None,NaT,NaT,None,None,None,None
2,1684788,REHEMA,KHAMIS ...,MOMBASA,102837,1955-09-30 00:00:00+02:45,...,F,0001-01-01 00:00:00+02:27,ALI,0307304,2016-01-01 00:00:00+03:00,,254733945631,MASS,1,1,NaN,None,None,None,None,NaT,None,NaN,None,NaN,NaT,NaT,NaT,NaN,NaT,NaN,NaT,None,NaT,NaT,None,None,None,None
3,1684978,MAROKO,NYAMBEGA ...,NAKURU,103339,1969-01-01 00:00:00+03:00,...,M,0001-01-01 00:00:00+02:27,,8243746,2016-01-01 00:00:00+03:00,,254711408266,MASS,1,1,35374316.0,MAROKO NYAMBEGA,2000069662,DORMANT,None,2001-01-01,KES,0.00,NAKURU,103339.0,2020-03-05 07:01:14.217035,2020-03-04,2001-01-01,0.0,2001-01-01,0.0,2001-01-01,True,2014-06-10,2008-08-12,None,NAKURU,TRANSACTIONAL ACCOUNT-CA,TRANSACTIONAL ACCOUNT-CA
4,1685098,EMILY,LUVINDI ...,GILL HOUSE,103594,1987-03-09 00:00:00+03:00,EMMY_KUDESA@YAHOO.COM ...,F,0001-01-01 00:00:00+02:27,KADESA,25820861,2016-01-01 00:00:00+03:00,,254721400539,MASS,1,1,NaN,None,None,None,None,NaT,None,NaN,None,NaN,NaT,NaT,NaT,NaN,NaT,NaN,NaT,None,NaT,NaT,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892708,1726163,GEOFFREY,NKUUBI ...,MERU,171148,1989-11-03 00:00:00+03:00,...,M,0001-01-01 00:00:00+02:27,KIOGORA,29018973,2016-01-01 00:00:00+03:00,,254707920462,MASS,1,1,35491205.0,GEOFFREY NKUUBI,2000109917,DORMANT,None,2001-01-01,KES,54.95,MERU,171148.0,2020-03-05 07:01:29.164176,2020-03-04,2001-01-01,0.0,2001-01-01,0.0,2001-01-01,True,2015-04-17,2015-03-10,None,MERU,CROSS OVER SAVING ACCOUNT,CROSS OVER SAVING ACCOUNT
892709,1726565,PATRICK,KIUNYU ...,MERU,171666,1982-04-27 00:00:00+03:00,CHATQNYU@YAHOO.COM ...,M,0001-01-01 00:00:00+02:27,NJOROGE,22442138,2016-01-01 00:00:00+03:00,,,MASS,1,1,NaN,None,None,None,None,NaT,None,NaN,None,NaN,NaT,NaT,NaT,NaN,NaT,NaN,NaT,None,NaT,NaT,None,None,None,None
892710,1727314,BRIAN,NTHUMBI ...,MOMBASA,172690,2011-04-04 00:00:00+03:00,PHNTHUMBI@GMAIL.COM ...,M,0001-01-01 00:00:00+02:27,SAMMY,0251108681,2016-01-01 00:00:00+03:00,,,MASS,1,1,35271143.0,BRIAN NTHUMBI,2000107266,DORMANT,None,2001-01-01,KES,513.45,MOMBASA,172690.0,2020-03-05 07:01:01.206511,2020-03-04,2001-01-01,0.0,2001-01-01,0.0,2001-01-01,True,2018-02-19,2014-11-18,None,MOMBASA,TREASURE SAVINGS ACCOUNT,TREASURE SAVINGS ACCOUNT
892711,1727790,BENARD,MOCHAMA ...,ELDORET,173330,1974-01-01 00:00:00+03:00,...,M,0001-01-01 00:00:00+02:27,MOKUA,11700558,2016-01-01 00:00:00+03:00,,254723079967,MASS,1,1,35247167.0,BENARD MOCHAMA,2000107609,DORMANT,None,2001-01-01,KES,343.44,ELDORET,173330.0,2020-03-05 07:00:58.115887,2020-03-04,2001-01-01,0.0,2001-01-01,0.0,2001-01-01,True,2014-12-02,2014-12-01,None,ELDORET,CROSS OVER SAVING ACCOUNT,CROSS OVER SAVING ACCOUNT


In [32]:
customers_dump.columns

Index(['id', 'firstname', 'lastname', 'branch', 'cust_id', 'dob', 'email',
       'gender', 'last_updated_date', 'middlename', 'national_id', 'open_date',
       'other_phone_numbers', 'phone_number', 'segment', 'status', 'type',
       'id_2', 'account_name', 'account_no', 'account_status', 'changed_by',
       'close_date', 'currency', 'current_balance', 'current_branch',
       'cust_id_2', 'date_created', 'eom_date', 'expiry_date', 'interest',
       'interest_end_date', 'interest_rate', 'interest_start_date',
       'is_transacting_account', 'last_transaction_date', 'open_date_2',
       'opened_by', 'opening_branch', 'product_type', 'type_2'],
      dtype='object')

In [33]:
customers_dump['dob'] = pd.to_datetime(customers_dump['dob'],utc=True,infer_datetime_format=True,errors = 'coerce' ) #, utc=True, errors='coerce' , errors='coerce'

customers_dump['dob']


0        1980-04-12 21:00:00+00:00
1        1970-12-31 21:00:00+00:00
2        1955-09-29 21:15:00+00:00
3        1968-12-31 21:00:00+00:00
4        1987-03-08 21:00:00+00:00
                    ...           
892708   1989-11-02 21:00:00+00:00
892709   1982-04-26 21:00:00+00:00
892710   2011-04-03 21:00:00+00:00
892711   1973-12-31 21:00:00+00:00
892712   1987-06-25 21:00:00+00:00
Name: dob, Length: 892713, dtype: datetime64[ns, UTC]

In [34]:
customers_dump['open_date'] = pd.to_datetime(customers_dump['open_date'],utc=True, infer_datetime_format=True ,errors='coerce') #, errors='coerce'
customers_dump['open_date'] 

0        2015-12-31 21:00:00+00:00
1        2015-12-31 21:00:00+00:00
2        2015-12-31 21:00:00+00:00
3        2015-12-31 21:00:00+00:00
4        2015-12-31 21:00:00+00:00
                    ...           
892708   2015-12-31 21:00:00+00:00
892709   2015-12-31 21:00:00+00:00
892710   2015-12-31 21:00:00+00:00
892711   2015-12-31 21:00:00+00:00
892712   2015-12-31 21:00:00+00:00
Name: open_date, Length: 892713, dtype: datetime64[ns, UTC]

In [35]:
import datetime
import pytz
my_date = datetime.datetime.now(pytz.timezone('UTC'))
my_date

datetime.datetime(2020, 3, 5, 5, 43, 40, 56893, tzinfo=<UTC>)

data_process = customers_dump.groupby(['cust_id','account_no']).agg({
    'dob': lambda x: (my_date - x.max()),
    'open_date': lambda x: (my_date - x.max())
})

data_process.rename(columns={'dob': 'age', 
                         'open_date': 'age_bank', 
                         }, inplace=True)

data_process

#data_process.age
# data_process.age
#pd.to_numeric(data_process.age.dt.days, downcast='integer')
#data_process.age = pd.to_numeric(data_process.age.dt.days, downcast='integer')
#data_process.age
#df['tdColumn'] = 
#df['tdColumn'].dt.days.astype('int16')
data_process.age = data_process.age.dt.days
data_process.age_bank = data_process.age_bank.dt.days

data_process.age_bank = data_process.age_bank.dt.days

data_process = customers_dump.groupby(['cust_id']).agg({
    'dob': lambda x: (NOW - x.max()).days,
    'open_date': lambda x: (NOW - x.max()).days
})

data_process.rename(columns={'dob': 'age', 
                         'open_date': 'age_bank', 
                         }, inplace=True)

customers_dump.groupby(['account_no']).agg({
    'dob': lambda x: (my_date-x.max()).days
})

data_process

data_process = pd.DataFrame(data_process)
data_process.reset_index(inplace=True)

data_process.columns

customers_dump.columns

data_process.age = pd.to_numeric(data_process.age, downcast='float')
data_process.age_bank = pd.to_numeric(data_process.age_bank , downcast='float')

In [36]:
customers_dump['age'] = my_date-customers_dump.dob
customers_dump['age_bank'] = my_date-customers_dump.open_date
customers_dump.age = customers_dump.age.dt.days
customers_dump.age_bank = customers_dump.age_bank.dt.days
customers_dump

,id,firstname,lastname,branch,cust_id,dob,email,gender,last_updated_date,middlename,national_id,open_date,other_phone_numbers,phone_number,segment,status,type,id_2,account_name,account_no,account_status,changed_by,close_date,currency,current_balance,current_branch,cust_id_2,date_created,eom_date,expiry_date,interest,interest_end_date,interest_rate,interest_start_date,is_transacting_account,last_transaction_date,open_date_2,opened_by,opening_branch,product_type,type_2,age,age_bank
0,1684944,HUDSON,MARIGA ...,HEAD OFFICE,103265,1980-04-12 21:00:00+00:00,...,M,0001-01-01 00:00:00+02:27,GACHERU,21982968,2015-12-31 21:00:00+00:00,,254722342828,MASS,1,1,35740180.0,HUDSON MARIGA,2000068873,DORMANT,None,2001-01-01,KES,0.00,NYERI,103265.0,2020-03-05 07:02:01.575886,2020-03-04,2001-01-01,0.0,2001-01-01,0.0,2001-01-01,True,2014-06-10,2008-06-30,None,NYERI,TRANSACTIONAL ACCOUNT-CA,TRANSACTIONAL ACCOUNT-CA,14571.0,1525.0
1,1684570,RICHARD,OMWANCHA ...,REHANI,102361,1970-12-31 21:00:00+00:00,...,M,0001-01-01 00:00:00+02:27,NYAMARI,10929252,2015-12-31 21:00:00+00:00,,254723291492,MASS,1,1,NaN,None,None,None,None,NaT,None,NaN,None,NaN,NaT,NaT,NaT,NaN,NaT,NaN,NaT,None,NaT,NaT,None,None,None,None,17961.0,1525.0
2,1684788,REHEMA,KHAMIS ...,MOMBASA,102837,1955-09-29 21:15:00+00:00,...,F,0001-01-01 00:00:00+02:27,ALI,0307304,2015-12-31 21:00:00+00:00,,254733945631,MASS,1,1,NaN,None,None,None,None,NaT,None,NaN,None,NaN,NaT,NaT,NaT,NaN,NaT,NaN,NaT,None,NaT,NaT,None,None,None,None,23533.0,1525.0
3,1684978,MAROKO,NYAMBEGA ...,NAKURU,103339,1968-12-31 21:00:00+00:00,...,M,0001-01-01 00:00:00+02:27,,8243746,2015-12-31 21:00:00+00:00,,254711408266,MASS,1,1,35374316.0,MAROKO NYAMBEGA,2000069662,DORMANT,None,2001-01-01,KES,0.00,NAKURU,103339.0,2020-03-05 07:01:14.217035,2020-03-04,2001-01-01,0.0,2001-01-01,0.0,2001-01-01,True,2014-06-10,2008-08-12,None,NAKURU,TRANSACTIONAL ACCOUNT-CA,TRANSACTIONAL ACCOUNT-CA,18691.0,1525.0
4,1685098,EMILY,LUVINDI ...,GILL HOUSE,103594,1987-03-08 21:00:00+00:00,EMMY_KUDESA@YAHOO.COM ...,F,0001-01-01 00:00:00+02:27,KADESA,25820861,2015-12-31 21:00:00+00:00,,254721400539,MASS,1,1,NaN,None,None,None,None,NaT,None,NaN,None,NaN,NaT,NaT,NaT,NaN,NaT,NaN,NaT,None,NaT,NaT,None,None,None,None,12050.0,1525.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892708,1726163,GEOFFREY,NKUUBI ...,MERU,171148,1989-11-02 21:00:00+00:00,...,M,0001-01-01 00:00:00+02:27,KIOGORA,29018973,2015-12-31 21:00:00+00:00,,254707920462,MASS,1,1,35491205.0,GEOFFREY NKUUBI,2000109917,DORMANT,None,2001-01-01,KES,54.95,MERU,171148.0,2020-03-05 07:01:29.164176,2020-03-04,2001-01-01,0.0,2001-01-01,0.0,2001-01-01,True,2015-04-17,2015-03-10,None,MERU,CROSS OVER SAVING ACCOUNT,CROSS OVER SAVING ACCOUNT,11080.0,1525.0
892709,1726565,PATRICK,KIUNYU ...,MERU,171666,1982-04-26 21:00:00+00:00,CHATQNYU@YAHOO.COM ...,M,0001-01-01 00:00:00+02:27,NJOROGE,22442138,2015-12-31 21:00:00+00:00,,,MASS,1,1,NaN,None,None,None,None,NaT,None,NaN,None,NaN,NaT,NaT,NaT,NaN,NaT,NaN,NaT,None,NaT,NaT,None,None,None,None,13827.0,1525.0
892710,1727314,BRIAN,NTHUMBI ...,MOMBASA,172690,2011-04-03 21:00:00+00:00,PHNTHUMBI@GMAIL.COM ...,M,0001-01-01 00:00:00+02:27,SAMMY,0251108681,2015-12-31 21:00:00+00:00,,,MASS,1,1,35271143.0,BRIAN NTHUMBI,2000107266,DORMANT,None,2001-01-01,KES,513.45,MOMBASA,172690.0,2020-03-05 07:01:01.206511,2020-03-04,2001-01-01,0.0,2001-01-01,0.0,2001-01-01,True,2018-02-19,2014-11-18,None,MOMBASA,TREASURE SAVINGS ACCOUNT,TREASURE SAVINGS ACCOUNT,3258.0,1525.0
892711,1727790,BENARD,MOCHAMA ...,ELDORET,173330,1973-12-31 21:00:00+00:00,...,M,0001-01-01 00:00:00+02:27,MOKUA,11700558,2015-12-31 21:00:00+00:00,,254723079967,MASS,1,1,35247167.0,BENARD MOCHAMA,2000107609,DORMANT,None,2001-01-01,KES,343.44,ELDORET,173330.0,2020-03-05 07:00:58.115887,2020-03-04,2001-01-01,0.0,2001-01-01,0.0,2001-01-01,True,2014-12-02,2014-12-01,None,ELDORET,CROSS OVER SAVI

In [37]:
customers_dump = customers_dump[[ 'firstname', 'lastname', 'branch', 'cust_id', 'dob', 'email',
       'gender',  'middlename', 'national_id', 'open_date',
       'other_phone_numbers', 'phone_number', 'segment', 'status', 'type',
        'account_name', 'account_no', 'account_status', 
       'close_date', 'currency', 'current_balance', 'current_branch',
        'eom_date', 'expiry_date', 'interest',
       'is_transacting_account', 
       'opened_by', 'opening_branch', 'product_type',  'age',
       'age_bank']]
customers_dump

,firstname,lastname,branch,cust_id,dob,email,gender,middlename,national_id,open_date,other_phone_numbers,phone_number,segment,status,type,account_name,account_no,account_status,close_date,currency,current_balance,current_branch,eom_date,expiry_date,interest,is_transacting_account,opened_by,opening_branch,product_type,age,age_bank
0,HUDSON,MARIGA ...,HEAD OFFICE,103265,1980-04-12 21:00:00+00:00,...,M,GACHERU,21982968,2015-12-31 21:00:00+00:00,,254722342828,MASS,1,1,HUDSON MARIGA,2000068873,DORMANT,2001-01-01,KES,0.00,NYERI,2020-03-04,2001-01-01,0.0,True,None,NYERI,TRANSACTIONAL ACCOUNT-CA,14571.0,1525.0
1,RICHARD,OMWANCHA ...,REHANI,102361,1970-12-31 21:00:00+00:00,...,M,NYAMARI,10929252,2015-12-31 21:00:00+00:00,,254723291492,MASS,1,1,None,None,None,NaT,None,NaN,None,NaT,NaT,NaN,None,None,None,None,17961.0,1525.0
2,REHEMA,KHAMIS ...,MOMBASA,102837,1955-09-29 21:15:00+00:00,...,F,ALI,0307304,2015-12-31 21:00:00+00:00,,254733945631,MASS,1,1,None,None,None,NaT,None,NaN,None,NaT,NaT,NaN,None,None,None,None,23533.0,1525.0
3,MAROKO,NYAMBEGA ...,NAKURU,103339,1968-12-31 21:00:00+00:00,...,M,,8243746,2015-12-31 21:00:00+00:00,,254711408266,MASS,1,1,MAROKO NYAMBEGA,2000069662,DORMANT,2001-01-01,KES,0.00,NAKURU,2020-03-04,2001-01-01,0.0,True,None,NAKURU,TRANSACTIONAL ACCOUNT-CA,18691.0,1525.0
4,EMILY,LUVINDI ...,GILL HOUSE,103594,1987-03-08 21:00:00+00:00,EMMY_KUDESA@YAHOO.COM ...,F,KADESA,25820861,2015-12-31 21:00:00+00:00,,254721400539,MASS,1,1,None,None,None,NaT,None,NaN,None,NaT,NaT,NaN,None,None,None,None,12050.0,1525.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892708,GEOFFREY,NKUUBI ...,MERU,171148,1989-11-02 21:00:00+00:00,...,M,KIOGORA,29018973,2015-12-31 21:00:00+00:00,,254707920462,MASS,1,1,GEOFFREY NKUUBI,2000109917,DORMANT,2001-01-01,KES,54.95,MERU,2020-03-04,2001-01-01,0.0,True,None,MERU,CROSS OVER SAVING ACCOUNT,11080.0,1525.0
892709,PATRICK,KIUNYU ...,MERU,171666,1982-04-26 21:00:00+00:00,CHATQNYU@YAHOO.COM ...,M,NJOROGE,22442138,2015-12-31 21:00:00+00:00,,,MASS,1,1,None,None,None,NaT,None,NaN,None,NaT,NaT,NaN,None,None,None,None,13827.0,1525.0
892710,BRIAN,NTHUMBI ...,MOMBASA,172690,2011-04-03 21:00:00+00:00,PHNTHUMBI@GMAIL.COM ...,M,SAMMY,0251108681,2015-12-31 21:00:00+00:00,,,MASS,1,1,BRIAN NTHUMBI,2000107266,DORMANT,2001-01-01,KES,513.45,MOMBASA,2020-03-04,2001-01-01,0.0,True,None,MOMBASA,TREASURE SAVINGS ACCOUNT,3258.0,1525.0
892711,BENARD,MOCHAMA ...,ELDORET,173330,1973-12-31 21:00:00+00:00,...,M,MOKUA,11700558,2015-12-31 21:00:00+00:00,,254723079967,MASS,1,1,BENARD MOCHAMA,2000107609,DORMANT,2001-01-01,KES,343.44,ELDORET,2020-03-04,2001-01-01,0.0,True,None,ELDORET,CROSS OVER SAVING ACCOUNT,16865.0,1525.0


In [41]:
balance_data = pd.read_excel("/Users/allan.aswani/Downloads/calculation/average_Balance-Dec-2020.xlsx")#turnovers_from_sep_to_feb.xlsx


In [ ]:
turnover_data = pd.read_excel("/Users/allan.aswani/Downloads/calculation/turnovers_from_sep_to_feb.xlsx")

In [ ]:
balance_data

In [ ]:
turnover_data.AVERAGE_TOTAL_CREDT = turnover_data.AVERAGE_TOTAL_CREDT.astype('int64')

In [ ]:
balance_data.ACC_NUM = balance_data.ACC_NUM.astype('int64')


In [ ]:
turnover_data.ACCOUNTNUMBER = turnover_data.ACCOUNTNUMBER.astype('int64')

In [ ]:
turnover_data = turnover_data[['ACCOUNTNUMBER',  'ID_NO', 'AVERAGE_TOTAL_CREDT']]

In [ ]:
customers_dump.columns

In [ ]:
customers_dump.account_no = customers_dump.account_no.replace(to_replace='None', value=np.nan).dropna().astype('int64')

balance_data = balance_data.merge(data_process, left_on="CUST_CIF", right_on="cust_id")

In [ ]:
balance_data = balance_data.merge(customers_dump, left_on="ACC_NUM",right_on="account_no" ,how='left')

In [ ]:
balance_data

In [ ]:
turnover_data

In [ ]:
balance_data.ACC_NUM = balance_data.ACC_NUM.astype('int64')

In [ ]:
turnover_data.ACCOUNTNUMBER = turnover_data.ACCOUNTNUMBER.astype('int64')

In [ ]:
balance_data = balance_data.merge(turnover_data, left_on="ACC_NUM",right_on="ACCOUNTNUMBER" ,how='left')

In [ ]:
balance_data

In [ ]:
turnover_data

In [ ]:
balance_data.ACC_NUM = balance_data.ACC_NUM.astype('int64')

In [ ]:
rfmTable.columns

In [ ]:
rfmTable.account_number = rfmTable.account_number.astype('int64')

In [ ]:
balance_data = balance_data.merge(rfmTable, left_on="ACC_NUM", right_on="account_number",how='left')

In [ ]:
#turnover_data = turnover_data.merge(rfmTable, left_on="ACC_NUM", right_on="account_number")

In [ ]:
balance_data

In [ ]:
loan_data = pd.read_csv("/Users/allan.aswani/Downloads/calculation/dean_data_final/loan_data_dump_4_feb_2020.csv")


In [ ]:
loan_data_turnover = pd.read_excel("/Users/allan.aswani/Downloads/calculation/repayment_data_03_03_2020.xlsx")

In [ ]:
loan_data

In [ ]:
loan_data_turnover

In [ ]:
loan_data

In [ ]:
loan_data['status_score']=loan_data.status.apply(status_category)
loan_data.head(5)

In [ ]:
loan_data_turnover['status_score']=loan_data_turnover.status.apply(status_category)
loan_data_turnover.head(5)

In [ ]:
loan_data_1 = loan_data.groupby(['CUST_ID','DEPOSIT_ACCOUNT']).agg({
    'LOAN_ACCOUNT': lambda x: x.count(),
    'INSTANT_LOAN_AMOUNT': lambda x: x.mean(),
    'status_score': lambda x:x.sum()
})

loan_data_1.rename(columns={'LOAN_ACCOUNT': 'count_of_loans_taken', 
                         'INSTANT_LOAN_AMOUNT': 'mean_loan_balances', 
                         }, inplace=True)

In [ ]:
loan_data_2 = loan_data_turnover.groupby(['CUST_ID']).agg({
    'ACCOUNT_NUMBER': lambda x: x.count(),
    'BALANCE': lambda x: x.mean(),
    'status_score': lambda x:x.sum()
})
loan_data_2.rename(columns={'ACCOUNT_NUMBER': 'count_of_loans_taken', 
                         'BALANCE': 'sum_current_outstadnging_loan_balances', 
                         }, inplace=True)

In [ ]:
loan_data_2

In [ ]:
loan_data_1 = pd.DataFrame(loan_data_1)
loan_data_1.reset_index( inplace=True)
loan_data_1

In [ ]:
loan_data_2 = pd.DataFrame(loan_data_2)
loan_data_2.reset_index( inplace=True)
loan_data_2

In [ ]:
loan_data_2

In [ ]:
#loan_data_turnover = loan_data_turnover.merge(loan_data_2, left_on='CUST_ID', right_on='CUST_ID', how='left' )
#loan_data_turnover
balance_data

In [ ]:
balance_data = balance_data.merge(loan_data_2, left_on='CUST_CIF', right_on='CUST_ID', how='left' )
balance_data

In [ ]:
balance_data.monetary_value

In [ ]:
balance_data

In [ ]:
balance_data['age_score'] = balance_data.age.apply(age_score)

In [ ]:
balance_data['age_of_acccount_in_bank_score'] = balance_data.age_bank.apply(age_of_acccount_in_bank_score)

In [ ]:
balance_data['recency_score'] = balance_data.recency.apply(recency_score)

In [ ]:
balance_data['frequency_score'] = balance_data.frequency.apply(frequency_score)

In [ ]:
balance_data['transaction_amount_sum_score'] = balance_data.monetary_value.apply(transaction_amount_sum_score)

In [ ]:
balance_data['status_score'] = balance_data['status_score'].apply(status_score)

In [ ]:
balance_data['count_of_loans_taken_score'] = balance_data.count_of_loans_taken.apply(count_of_loans_taken_score)

In [ ]:
balance_data[balance_data.status_score >0]

In [ ]:
balance_data['total_gross_score'] =  balance_data[['age_score','age_of_acccount_in_bank_score','recency_score', 'frequency_score', 'count_of_loans_taken_score','transaction_amount_sum_score','status_score']].sum(axis =1)





In [ ]:
balance_data

In [ ]:
balance_data[balance_data.account_number ==2000001228]

In [ ]:
balance_data['multiplier_score'] =  balance_data.total_gross_score.apply(limit_mutiplier)

In [ ]:
balance_data['gross_limit_from_average_balance'] = balance_data[['DEC_18_BAL', 'JAN_BAL', 'FEB_BAL', 'MAR_BAL', 'APR_BAL', 'MAY_BAL', 'JUN_BAL', 'JUL_BAL', 'AUG_BAL', 'SEP_BAL','OCT_BAL', 'NOV_BAL', 'DEC_19_BAL']].mean(axis =1)

In [ ]:
balance_data

In [ ]:
balance_data.AVERAGE_TOTAL_CREDT.unique()

In [ ]:
balance_data['Limit_1']  = balance_data['gross_limit_from_average_balance'] * balance_data['multiplier_score'] 




In [ ]:
balance_data['limit_ratified_max_30K_min_1k'] = balance_data['Limit_1'].apply(limit_ratified)

In [ ]:
balance_data['limit_ratified_max_30K_min_1k'].sum()

In [ ]:
balance_data['limit_ratified_max_30K_min_1k'].count()

balance_data[balance_data.Limit_2>1000]

balance_data[balance_data['total_gross_score']>0]

conn = psycopg2.connect(host="128.2.1.25",database="datawarehouse", user="datawarehouse", password="P@ssw0rd")

customers_1 = '''SELECT "public"."accounts"."id" AS "id", "public"."accounts"."account_name" AS "account_name", "public"."accounts"."account_no" AS "account_no", "public"."accounts"."account_status" AS "account_status", "public"."accounts"."changed_by" AS "changed_by", "public"."accounts"."close_date" AS "close_date", "public"."accounts"."currency" AS "currency", "public"."accounts"."current_balance" AS "current_balance", "public"."accounts"."current_branch" AS "current_branch", "public"."accounts"."cust_id" AS "cust_id", "public"."accounts"."date_created" AS "date_created", "public"."accounts"."eom_date" AS "eom_date", "public"."accounts"."expiry_date" AS "expiry_date", "public"."accounts"."interest" AS "interest", "public"."accounts"."interest_end_date" AS "interest_end_date", "public"."accounts"."interest_rate" AS "interest_rate", "public"."accounts"."interest_start_date" AS "interest_start_date", "public"."accounts"."is_transacting_account" AS "is_transacting_account", "public"."accounts"."last_transaction_date" AS "last_transaction_date", "public"."accounts"."open_date" AS "open_date", "public"."accounts"."opened_by" AS "opened_by", "public"."accounts"."opening_branch" AS "opening_branch", "public"."accounts"."product_type" AS "product_type", "public"."accounts"."type" AS "type", "Customers"."id" AS "id_2", "Customers"."firstname" AS "firstname", "Customers"."lastname" AS "lastname", "Customers"."branch" AS "branch", "Customers"."cust_id" AS "cust_id_2", "Customers"."dob" AS "dob", "Customers"."email" AS "email", "Customers"."gender" AS "gender", "Customers"."last_updated_date" AS "last_updated_date", "Customers"."middlename" AS "middlename", "Customers"."national_id" AS "national_id", "Customers"."open_date" AS "open_date_2", "Customers"."other_phone_numbers" AS "other_phone_numbers", "Customers"."phone_number" AS "phone_number", "Customers"."segment" AS "segment", "Customers"."status" AS "status", "Customers"."type" AS "type_2"
FROM "public"."accounts"
LEFT JOIN "public"."customers" "Customers" ON "public"."accounts"."cust_id" = "Customers"."cust_id"'''
customers_1_2 = pd.read_sql_query(customers_1 , conn)
customers_1_2

customers_1_2 = pd.read_csv('/Users/allan.aswani/Downloads/calculation/customers_1_2_12_feb_2020.csv')

customers_1_2.account_no = customers_1_2.account_no.astype('int64')

balance_data = balance_data.merge(customers_1_2, left_on='ACC_NUM', right_on='account_no', how='left' )
balance_data

In [ ]:
balance_data

In [ ]:
balance_data['account_status_ratified'] = balance_data.account_status.apply(account_status_ratified)

In [ ]:
balance_data['joint_account_status_ratified'] = balance_data.type.apply(joint_account_status_ratified)

In [ ]:
balance_data.is_transacting_account.unique()

In [ ]:
balance_data['remove_non_transacting_accounts'] = balance_data.is_transacting_account.apply(remove_non_transacting_accounts)

In [ ]:
#balance_data['Limit_3']  = balance_data['remove_non_transacting_accounts'] * balance_data['account_status_ratified']*balance_data['joint_account_status_ratified'] *balance_data.limit_ratified_max_30K_min_1k

In [ ]:
balance_data

In [ ]:
balance_data['currency_ratified_score'] = balance_data.currency.apply(currency_ratified_score)
balance_data['remove_non_qualifying_products'] = balance_data.product_type.apply(remove_non_qualifying_products)
balance_data['Limit_3']  =balance_data['remove_non_qualifying_products']*balance_data['currency_ratified_score']* balance_data['remove_non_transacting_accounts'] * balance_data['account_status_ratified']*balance_data['joint_account_status_ratified'] *balance_data.limit_ratified_max_30K_min_1k
balance_data['Limit_3'] = round(balance_data['Limit_3'],-2)
balance_data


In [ ]:
balance_data

In [ ]:
write_off_accounts = pd.read_excel("/Users/allan.aswani/Downloads/calculation/write_off_accounts.xlsx") #,sep=',', error_bad_lines=False, index_col=False, dtype='unicode'
write_off_accounts

In [ ]:
write_off_accounts.PAYING_ACCOUNT = write_off_accounts.PAYING_ACCOUNT.astype('int64')


In [ ]:
write_off_accounts.columns
write_off_accounts

In [ ]:
write_off_accounts_1 = write_off_accounts[['PAYING_ACCOUNT','write_off']]
write_off_accounts_1

In [ ]:
#balance_data.national_id.astype('int64')

balance_data.ACC_NUM = balance_data.ACC_NUM.astype('int64')

In [ ]:
balance_data = balance_data.merge(write_off_accounts_1, left_on="ACC_NUM", right_on="PAYING_ACCOUNT", how='left' )
balance_data

In [ ]:
#send_to_tu_balance_data = send_to_tu_balance_data[send_to_tu_balance_data.total_gross_score>0]   

In [ ]:
#send_to_tu_balance_data = balance_data[balance_data.Limit_4<500]

In [ ]:
#bool_series = data["First Name"].duplicated(keep = False) 

In [ ]:
balance_data['remove_write_off_accounts'] = balance_data.write_off.apply(remove_write_off_accounts)

In [ ]:
balance_data['Limit_4'] = balance_data['Limit_3']*balance_data['remove_write_off_accounts']

In [ ]:
balance_data

In [ ]:
balance_data['phone_number_check'] = balance_data.phone_number.astype('str')
balance_data['phonenumber_length'] =balance_data.phone_number.str.len()
balance_data['phone_number_multiplier'] = balance_data.phonenumber_length.apply(phone_number_check)
balance_data['Limit_5'] = balance_data['phone_number_multiplier']*balance_data['Limit_4']
balance_data

In [ ]:
balance_data['removing_unqualifying_segments'] = balance_data.segment.apply(removing_unqualifying_segments)
balance_data['Limit_6'] = balance_data.Limit_5*balance_data['removing_unqualifying_segments']
balance_data

In [ ]:
balance_data['dplicated_cust_cif'] = balance_data.duplicated('CUST_CIF')
balance_data['dplicated_cust_cif'] = balance_data['dplicated_cust_cif'].replace({True: 0, False: 1})
balance_data['dplicated_national_id'] = balance_data.duplicated('national_id')
balance_data['dplicated_national_id']= balance_data['dplicated_national_id'].replace({True: 0, False: 1})
balance_data['dplicated_phone_number'] = balance_data.duplicated('phone_number')
balance_data['dplicated_phone_number']= balance_data['dplicated_phone_number'].replace({True: 0, False: 1})
balance_data['Limit_7'] = balance_data.Limit_6*balance_data['dplicated_cust_cif'] * balance_data['dplicated_national_id']* balance_data['dplicated_phone_number']
balance_data

In [ ]:
balance_data.columns


In [ ]:
balance_data.head()

In [ ]:
balance_data['tu_fist_check'] = balance_data[[ 'account_status_ratified', 'joint_account_status_ratified', 'remove_non_transacting_accounts',
       'currency_ratified_score', 'remove_non_qualifying_products',  'remove_write_off_accounts', 
      'phone_number_multiplier', 'removing_unqualifying_segments', 
        'dplicated_phone_number']].mean(axis =1)   #'dplicated_cust_cif', 'dplicated_national_id',

balance_data[balance_data.tu_fist_check==1]

In [ ]:
send_to_tu_balance_data = balance_data[balance_data.tu_fist_check==1]
send_to_tu_balance_data = send_to_tu_balance_data[send_to_tu_balance_data.total_gross_score>=-30]
send_to_tu_balance_data = send_to_tu_balance_data[send_to_tu_balance_data.total_gross_score<=30]
send_to_tu_balance_data

In [ ]:
send_to_tu_balance_data[send_to_tu_balance_data.gross_limit_from_average_balance>0]

In [ ]:
send_to_tu_balance_data = balance_data[balance_data.total_gross_score>0]
send_to_tu_balance_data


In [ ]:
send_to_tu_balance_data

In [ ]:
send_to_tu_balance_data['send_to_tu_check']    = send_to_tu_balance_data.remove_non_qualifying_products*send_to_tu_balance_data.remove_write_off_accounts *send_to_tu_balance_data.currency_ratified_score * send_to_tu_balance_data.remove_non_transacting_accounts *send_to_tu_balance_data.account_status_ratified

send_to_tu_balance_data = send_to_tu_balance_data[send_to_tu_balance_data.send_to_tu_check==1]
send_to_tu_balance_data

In [ ]:
send_to_tu_balance_data = send_to_tu_balance_data[send_to_tu_balance_data.monetary_value>0]
send_to_tu_balance_data

In [ ]:
send_to_tu_balance_data = send_to_tu_balance_data[send_to_tu_balance_data.gross_limit_from_average_balance <1100]
send_to_tu_balance_data

In [ ]:
send_to_tu_balance_data['removing_unqualifying_segments'] = send_to_tu_balance_data.segment.apply(removing_unqualifying_segments)
send_to_tu_balance_data.Limit_4 = send_to_tu_balance_data.Limit_4*send_to_tu_balance_data['removing_unqualifying_segments']
send_to_tu_balance_data



In [ ]:
#bool_series = data["First Name"].duplicated(keep = False) 
#balance_data['duplicated_phone_numbers'] balance_data.phone_number.duplicated(keep=False)

In [ ]:
balance_data_1 = balance_data[balance_data.Limit_4>=1000]
balance_data_1

In [ ]:
balance_data_1 = balance_data_1 [balance_data_1.current_balance>0]
balance_data_1 = balance_data_1[balance_data_1.Limit_4>1000]
balance_data_1 = balance_data_1.sort_values(by=['Limit_4'])
balance_data_1 = balance_data_1 [balance_data_1.Limit_4>0]
balance_data_1 

In [ ]:
#removing duplicate cif
balance_data_1 = balance_data_1.drop_duplicates(subset ="CUST_CIF", keep=False, inplace=False)
balance_data_1

In [ ]:
#removing duplicate national id
balance_data_1 = balance_data_1.drop_duplicates(subset ="national_id", keep=False, inplace=False)
balance_data_1

In [ ]:
#removing duplicate phone_number
balance_data_1 = balance_data_1.drop_duplicates(subset ="phone_number", keep=False, inplace=False)
balance_data_1

In [ ]:
balance_data_1['removing_unqualifying_segments'] = balance_data_1.segment.apply(removing_unqualifying_segments)
balance_data_1.Limit_4 = balance_data_1.Limit_4*balance_data_1['removing_unqualifying_segments']
balance_data_1

In [ ]:
balance_data_1 = balance_data_1 [balance_data_1.current_balance>0]
balance_data_1 = balance_data_1[balance_data_1.Limit_4>1000]
balance_data_1 = balance_data_1.sort_values(by=['Limit_4'])
balance_data_1 = balance_data_1 [balance_data_1.Limit_4>0]
balance_data_1 

In [ ]:
#str.startswith(str, beg=0,end=len(string))

#balance_data_1.phone_number = balance_data_1.phone_number.astype('str')


In [ ]:
export_csv = balance_data_1.to_csv (r'/Users/allan.aswani/Downloads/calculation/balance_data_19_feb_2020.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


In [ ]:
#rfmTable.to_csv (r'/Users/allan.aswani/Downloads/calculation/alfred_rfmTable_12_feb_2020.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [ ]:
#rfmTable

In [ ]:
#export_csv = customers_1_2.to_csv (r'/Users/allan.aswani/Downloads/calculation/customers_1_2_12_feb_2020.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


In [ ]:
#pd.read_csv('/Users/allan.aswani/Downloads/calculation/customers_1_2_12_feb_2020.csv')

In [ ]:
#export_csv = customers_dump.to_csv (r'/Users/allan.aswani/Downloads/calculation/customers_dump_data_18_feb_2020.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


In [ ]:
#balance_data_1
send_to_tu_balance_data

In [ ]:
export_csv = send_to_tu_balance_data.to_csv (r'/Users/allan.aswani/Downloads/calculation/send_to_tu_balance_data_20_feb_2020.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


In [ ]:
export_csv = balance_data.to_csv (r'/Users/allan.aswani/Downloads/calculation/balance_data_20_feb_2020_tonney.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


In [ ]:
final_apploaded = pd.read_csv('/Users/allan.aswani/Downloads/calculation/existing_upload_results_final_list.csv')
#loan_data.head(5)
#for row in loan_data:
 #   print(row) 
final_apploaded

In [ ]:
final_apploaded.columns

In [ ]:
final_apploaded.rename(columns={'Id_no': 'final_national_id',
                                 'Account': 'final_account', 
                                 'Msisdn':'final_msisdn', 
                                 'Limit':'final_limit',
                                 'Date Created' :'final_date_created',
                                 'Date Uploaded':'final_date_uploaded', 
                                'Upload response':'final_upload_response'  , 
                                    'score_card':'score_card', 
                         }, inplace=True)

In [ ]:
final_apploaded.columns

In [ ]:
final_apploaded = final_apploaded[['final_national_id',  'final_account',
       'final_msisdn', 'final_limit', 'final_date_created',
       'final_date_uploaded', 'final_upload_response', 'score_card']]

In [ ]:
final_apploaded.final_account = final_apploaded.final_account.astype('int64')

In [ ]:
balance_data = balance_data.merge(final_apploaded, left_on="ACC_NUM", right_on="final_account", how='left' )
balance_data

In [ ]:
balance_data

In [ ]:
balance_data[balance_data.score_card == 'ETB']

In [ ]:
write_balance_data = balance_data[balance_data.score_card != 'ETB']

In [ ]:
write_balance_data_1 = write_balance_data[write_balance_data.tu_fist_check == 1.]
write_balance_data_1

In [ ]:
write_balance_data_2 = write_balance_data_1[write_balance_data_1.current_balance>0]
write_balance_data_2

In [ ]:
write_balance_data_2 = write_balance_data_2[write_balance_data_2.gross_limit_from_average_balance>0]
write_balance_data_2

In [ ]:
write_balance_data.columns


write_balance_data = write_balance_data[[  'OPEN_DATE', 'BRN_CODE', 'PROD_ID',
       'CUSTOMER_SEGMENT', 'RM_CODE','DIASPORA_CHECK','firstname',
      'lastname','branch','cust_id','dob','email','gender',
        'status','type','account_name','account_status','currency',
        'current_balance','current_branch','opening_branch',
        'product_type','age','age_bank','account_number',
        'recency','frequency','monetary_value','count_of_loans_taken',
        'status_score','age_score',
        'age_of_acccount_in_bank_score','recency_score',
        'frequency_score','transaction_amount_sum_score',
        'count_of_loans_taken_score','total_gross_score',
        'multiplier_score','gross_limit_from_average_balance',
        'account_status_ratified','joint_account_status_ratified',
        'remove_non_transacting_accounts','currency_ratified_score',
        'remove_non_qualifying_products','write_off',
        'remove_write_off_accounts',
        'removing_unqualifying_segments',
         'Limit_7', 'tu_fist_check', 'final_national_id', 'final_account',
       'final_msisdn', 'final_limit', 'final_date_created',
       'final_date_uploaded', 'final_upload_response', 'score_card']]
write_balance_data 

In [ ]:
write_balance_data.rename(columns={                               
                                'OPEN_DATE': 'open_date', 
                                'BRN_CODE': 'brn_code', 
                                'PROD_ID': 'prod_id',
                                'CUSTOMER_SEGMENT': 'customer_segment', 
                                'RM_CODE': 'rm_code',
                         }, inplace=True)

In [ ]:
write_balance_data

In [ ]:
write_balance_data_1 =     write_balance_data[write_balance_data.current_balance>0]
write_balance_data_1

In [ ]:
write_balance_data_1 = write_balance_data_1[write_balance_data_1.total_gross_score>0]
write_balance_data_1

In [ ]:
write_balance_data_1 = write_balance_data_1[write_balance_data_1.tu_fist_check ==1. ]
write_balance_data_1

import sql  # the patched version (file is named sql.py)
sql.write_frame(write_balance_data, 'score_card_output', conn, flavor='postgresql')

write_balance_data.to_sql('score_card_output', con= conn, if_exists='append',flavor='postgresql')

write_balance_data.to_sql('score_card_output', conn, if_exists='append')

In [ ]:
write_balance_data_1 =     write_balance_data[write_balance_data.current_balance>0]
write_balance_data_1 = write_balance_data_1[write_balance_data_1.tu_fist_check ==1. ]
write_balance_data_1 = write_balance_data_1[write_balance_data_1.total_gross_score>0]
write_balance_data_1 = write_balance_data_1[write_balance_data_1.gross_limit_from_average_balance>0]
write_balance_data_1 = write_balance_data_1[write_balance_data_1.total_gross_score>0]
write_balance_data_1 = write_balance_data_1[write_balance_data_1.total_gross_score<30]
write_balance_data_1 

In [ ]:
write_balance_data_1.to_csv (r'/Users/allan.aswani/Downloads/calculation/write_balance_data_03_mar_2020_send_to_tu.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

